# 🎬 Análisis de Secciones No Oficiales del Festival de Cannes

Este notebook recopila datos desde la API del Festival de Cannes sobre secciones no oficiales entre 2015 y 2024, y destaca la participación de películas provenientes de España 🇪🇸, Francia 🇫🇷 y Estados Unidos 🇺🇸.

In [ ]:
import requests
import pandas as pd
import time

headers = {
    "User-Agent": "Mozilla/5.0"
}

# Identificadores de secciones no oficiales
secciones = {
    "Un Certain Regard": "un-certain-regard",
    "Out of Competition": "hors-competition",
    "Cinéfondation": "cinefondation",
    "Quinzaine des Réalisateurs": "quinzaine-des-realisateurs",
    "Semaine de la Critique": "semaine-de-la-critique"
}

años = list(range(2015, 2025))
datos_peliculas = []

for año in años:
    for nombre_seccion, id_seccion in secciones.items():
        url = f"https://www.festival-cannes.com/en/api/films/selection/{id_seccion}?year={año}"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            peliculas = response.json().get("films", [])
            for pelicula in peliculas:
                paises = [c["label"] for c in pelicula.get("countries", [])]
                paises_str = ", ".join(paises)

                datos_peliculas.append({
                    "Año": año,
                    "Sección": nombre_seccion,
                    "Título": pelicula.get("title", "").strip(),
                    "Director": pelicula.get("director", "").strip() if pelicula.get("director") else "",
                    "Duración": pelicula.get("duration"),
                    "Categoría": pelicula.get("category", {}).get("label", ""),
                    "Países": paises_str,
                    "España": any(p.lower() in ["españa", "spain"] for p in paises),
                    "Francia": any(p.lower() in ["francia", "france"] for p in paises),
                    "EEUU": any(p.lower() in ["estados unidos", "united states", "usa", "états-unis"] for p in paises),
                    "Enlace": f"https://www.festival-cannes.com/en/festival/films/{pelicula.get('id')}"
                })
            print(f"✅ {nombre_seccion} {año}: {len(peliculas)} películas.")
        else:
            print(f"❌ Error en {nombre_seccion} {año}: Código {response.status_code}")
        time.sleep(1)

df = pd.DataFrame(datos_peliculas)
df.to_excel("cannes_no_oficiales_con_paises.xlsx", index=False)
print("🎉 Datos guardados en 'cannes_no_oficiales_con_paises.xlsx'")